In [ ]:
import pickle
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import shuffle

path_list = [
    '/home/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Tuesday-WorkingHours.pcap_ISCX.csv',
    '/home/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Wednesday-workingHours.pcap_ISCX.csv',
    '/home/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    '/homme/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    '/home/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Friday-WorkingHours-Morning.pcap_ISCX.csv',
    '/home/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
    '/home/keshav/Desktop/CIC AWS 2018/cicids 2017/cicids data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
]


def reduce_data(data):
    label = list(data[' Label'].unique())
    data_count = data[' Label'].value_counts()
    attack_count = 0
    for count in data_count[1:]:
        attack_count = attack_count + count

    data1 = pd.DataFrame()
    for name in label:
        if name is label[0]:
            temp_data = data[data[' Label'] == name]
            temp_data = temp_data[:attack_count * 2]
            data1 = pd.concat([data1, temp_data], ignore_index=True)

        else:
            temp_data = data[data[' Label'] == name]
            data1 = pd.concat([data1, temp_data], ignore_index=True)

    return data1


def merge_data():
    data = pd.DataFrame()
    for path in path_list:
        # print(path)
        if path == path_list[1]:
            file = open('ddos_data.txt', 'rb')
            temp_data = pickle.load(file)
            reduced_data = reduce_data(temp_data)
            data = pd.concat([data, reduced_data], ignore_index=True)

        else:
            temp_data = pd.read_csv(path)
            reduced_data = reduce_data(temp_data)
            data = pd.concat([data, reduced_data], ignore_index=True)

        data = shuffle(data)

    return data


def get_data(data):
    column_list = data.columns
    info = data[column_list[-1]].value_counts()
    data_info = list(data[column_list[-1]].unique())
    label_length = len(data_info)
    attack_labels = data_info[1:label_length]

    # Appending malicious data to the dataframe.
    bruteforce_data = pd.DataFrame()
    for label in attack_labels:
        temp_data = data[data[column_list[-1]] == label]
        bruteforce_data = pd.concat([bruteforce_data, temp_data],
                                    ignore_index=True)

    # Appending benign data to the dataframe.
    benign_data_all = data[data[column_list[-1]] == data_info[0]]
    benign_data = benign_data_all[:len(bruteforce_data)]
    bruteforce_data = pd.concat([bruteforce_data, benign_data],
                                ignore_index=True)

    # Shuffling the dataset.
    bruteforce_data = shuffle(bruteforce_data)
    bruteforce_data.index = np.arange(0, len(bruteforce_data))

    # Dropping the destination host column.
    bruteforce_data = bruteforce_data.drop([column_list[0]], axis=1)

    # Splitting the dataset into X and Y.
    y_bruteforce = bruteforce_data[' Label']
    x_bruteforce = bruteforce_data.drop([' Label'], axis=1)

    # Handling infinite and nan values.
    x_bruteforce.replace([np.inf, -np.inf], np.nan, inplace=True)
    x_bruteforce.fillna(x_bruteforce.mean(), inplace=True)

    # Doing feature scaling.
    scaler = StandardScaler()
    x_bruteforce = scaler.fit_transform(x_bruteforce)
    x_bruteforce = pd.DataFrame(x_bruteforce)
    column_list = column_list.drop([column_list[0], column_list[-1]])
    x_bruteforce.columns = column_list

    # Label encoding.
    le = LabelEncoder()
    y_bruteforce = le.fit_transform(y_bruteforce)

    # Train dataset.
    x_train_bruteforce = x_bruteforce[:int(0.8 * (len(x_bruteforce)))]
    y_train_bruteforce = y_bruteforce[:int(0.8 * (len(y_bruteforce)))]

    # Test dataset.
    x_test_bruteforce = x_bruteforce[int(0.8 * (len(x_bruteforce))):]
    y_test_bruteforce = y_bruteforce[int(0.8 * (len(x_bruteforce))):]

    return x_train_bruteforce, y_train_bruteforce, x_test_bruteforce, y_test_bruteforce


def get_cic_data(data):
    file = open('final_data.txt', 'rb')
    data = pickle.load(file)
    x_train, y_train, x_test, y_test = get_data(data)
    return x_train, y_train, x_test, y_test


def divide_without_label(parts, X_train_full, y_train_full):
    each_part_number = int(len(X_train_full) / parts)
    list_x_train = []
    list_y_train = []

    number_list = []
    number_list.append(0)
    for x in range(1, parts + 1):
        number_list.append(each_part_number * x)

    for x in range(0, parts):
        data_x = X_train_full[number_list[x]:number_list[x + 1]]
        data_y = y_train_full[number_list[x]:number_list[x + 1]]
        list_x_train.append(data_x)
        list_y_train.append(data_y)

    return list_x_train, list_y_train


def main():
    data = merge_data()
    x_train, y_train, x_valid, y_valid = get_cic_data(data)
    x_data, y_data = divide_without_label(11, x_train, y_train)


if __name__ == "__main__":
    main()
